In [ ]:
# make histogram of REM predictions 
import random
from matplotlib import pyplot as plt
import numpy as np
import math
dataset = 'TEST' #log10(area).pa.ratio' #, 'tri.10m', 'slope', 'easting', 'northing', elev.km', ### 'All'
from google.colab import drive
drive.mount('/content/gdrive')
old_ls_file = # specify your path to "Copy of ls_data_SVM.1_28_2021.csv"
predictionfile =  # specify path to "Predictions." +dataset+ ".csv"
update_file = # specify path to "FINAL.ls_data_SVM.Table.Rem." +dataset+ ".Predictions.csv"  

with open(old_ls_file, 'r') as f:
  data = f.read()
  print(repr(data)[:100])
  print(repr(data)[-100:])
  print(len(data))
f.close()
lines = data.split('\n')
lines.pop()
print(len(lines))
print(lines[0])
print(lines[1])

headings = lines[0].split(',')
print(headings)

table =[]
for x in range(1, len(lines)):
  row = lines[x].split(',')
  if row[len(row)-1] != 'NA':
    for y in range(1, len(row)): row[y] = float(row[y])
    table.append([ row[0], row[15], row[7], row[8], row[9], row[3], math.log10(row[16]),row[5], row[17]/row[16], row[20] ])

youngraw, oldraw =[], [] 
for x in range(len(table)):
  if table[x][1] <= 5:
    youngraw.append(table[x])
  else: 
    table[x].append(0)
    oldraw.append(table[x])

numruns =100
with open(predictionfile, 'r') as d:
  predsraw = d.read()
  preds = predsraw.split(',')
  preds.pop()
d.close() 
print("Number of preds=", len(preds))

scores =[]
for x in range(len(preds)):
  scores.append(float(preds[x]) / numruns)
print('Min and max scores', min(scores), max(scores))
plt.title('Distribution of Predicted REM Scores')
plt.xlabel('Fraction of REM-positive Predictions')
plt.ylabel('Number of Sites')
plt.hist(scores, 10)
plt.show()

sum_no, sum_yes, remainder = 0,0,0
for x in range(len(scores)):
  if scores[x] <= 0.33: sum_no +=1
  elif scores[x] >= 0.67: sum_yes +=1
  else: remainder +=1
print('Total sites =', (sum_no + sum_yes + remainder), 'sum_no =', sum_no, 'sum_yes =', sum_yes, 'remainder =', remainder)

old_headings = [ 'ID', 'age', 'elev.km', 'ls.northing', 'ls.easting', 'mean.slope', 'log(surface.area)', 'mean.tri', 'tri.10m', 'pa.ratio', 'old.rem', 'svm.rem' ]

file_headings =''
for x in range(len(old_headings)-1):
  file_headings += old_headings[x] + ','
file_headings += old_headings[len(old_headings) -1] + '\n'
print(file_headings)

with open(update_file, 'w') as h:
  h.write(file_headings)
  for x in range(len(oldraw)):
    outline =''
    for y in range(len(oldraw[x])):
      outline += str(oldraw[x][y]) + ',' 
    outline += str(scores[x]) + '\n'
    h.write(outline)
    #print(outline)
h.close()                 

agree_old_tridrop =0
for x  in range(len(oldraw)):
  if (float(oldraw[x][len(oldraw[x])-1]) >=0.5 and float(scores[x]) >=0.5) or \
     (float(oldraw[x][len(oldraw[x])-1]) <0.5 and float(scores[x]) <0.5):\
    agree_old_tridrop += 1
  else: pass
print('...')
print('Ageement between Old vs. TriDrop:', agree_old_tridrop / len(oldraw))

for x in range(10):
  print(oldraw[x], scores[x])
